### A.9.2 Single-GPU training

Now that we are familiar with transferring tensors to the GPU, we can modify the training loop to run on a GPU. This step requires only changing three lines of code, as shown in the following listing.


In [6]:
import torch

# Creating a multilayer perceptron with two hidden layers

class NeuralNetwork(torch.nn.Module):
  def __init__(self, num_inputs, num_outputs):
    super().__init__()
    self.layers = torch.nn.Sequential(
        # First hidden layer
        torch.nn.Linear(num_inputs, 30),
        torch.nn.ReLU(),

        # Second hidden layer
        torch.nn.Linear(30, 20),
        torch.nn.ReLU(),

        # Output layer
        torch.nn.Linear(20, num_outputs)
    )

  def forward(self, x):
    logits = self.layers(x)
    return logits


# Creating a small toy dataset
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])

y_train = torch.tensor([0, 0, 0, 1, 1])

x_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],])

y_test = torch.tensor([0, 1])


In [7]:
# Defining a custom Dataset class
from torch.utils.data import Dataset

class ToyDataset(Dataset):
  def __init__(self, X, y):
    self.features = X
    self.labels = y

  def __getitem__(self, index):
    one_x = self.features[index]
    one_y = self.labels[index]
    return one_x, one_y

  def __len__(self):
    return self.labels.shape[0]


train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(x_test, y_test)

In [8]:

# Instantiating data loaders
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0)


test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

In [12]:
import torch.nn.functional as F

#  A training loop on a GPU
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)

device = torch.device('mps')
model = model.to(device)

optimiser = torch.optim.SGD(model.parameters(),
                            lr=0.5)

num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        features, labels = features.to(device), labels.to(device)
        logits = model(features)
        loss = F.cross_entropy(logits, labels)

        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx+1:03d}/{len(train_loader):03d}"
              f" | Train Loss: {loss:.2f}")

Epoch: 001/003 | Batch 001/003 | Train Loss: 0.75
Epoch: 001/003 | Batch 002/003 | Train Loss: 0.65
Epoch: 001/003 | Batch 003/003 | Train Loss: 0.42
Epoch: 002/003 | Batch 001/003 | Train Loss: 0.05
Epoch: 002/003 | Batch 002/003 | Train Loss: 0.13
Epoch: 002/003 | Batch 003/003 | Train Loss: 0.00
Epoch: 003/003 | Batch 001/003 | Train Loss: 0.01
Epoch: 003/003 | Batch 002/003 | Train Loss: 0.00
Epoch: 003/003 | Batch 003/003 | Train Loss: 0.02


Running the preceding code will output the following, similar to the results obtained on the CPU.

We can use `.to("cuda")` | `.to("mps")` instead of `device = torch.device("cuda")` | `device = torch.device('mps')`.
Transferring a tensor to `"cuda"` instead of `torch.device("cuda")` works as well and is shorter. We can also modify the statement, which will make the same code executable on a CPU if a GPU is not available. This is considered best practice when sharing PyTorch code:



In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In the case of the modified training loop here, we probably won’t see a speedup due to the memory transfer cost from CPU to GPU. However, we can expect a significant speedup when training deep neural networks, especially LLMs.